# Automated Classification of Argumentative Elements in Student Writing

## Tabla de Contenidos
1. [Introduction](#Introducción)
2. [1. Selección de Algoritmos](#1-Selección-de-Algoritmos)
3. [2. Preprocesamiento e Ingeniería de Características](#2-Preprocesamiento-e-Ingeniería-de-Características)
4. [3. Implementación de Modelos](#3-Implementación-de-Modelos)
5. [4. Evaluación y Comparación de Modelos](#4-Evaluación-y-Comparación-de-Modelos)
6. [5. Visualización y Comunicación de Resultados](#5-Visualización-y-Comunicación-de-Resultados)
7. [Conclusiones](#Conclusiones)
8. [Referencias](#Referencias)


## Introducción

El objetivo de este proyecto se basa en crear modelos de machine para clasificar los elementos argumentativos en los textos escritos por estudiantes como "efectivos", "adecuados" o "ineficaces". Esta clasificación automatizada tiene como objetivo proporcionar una mejor retroalimentación a los estudiantes, para así ellos puedan mejorar sus habilidades de escritura.

---

## 1. Selección de Algoritmos

### Investigación y Selección de Algoritmos

Para abordar el problema de clasificación de elementos argumentativos en textos de estudiantes, se seleccionaron los siguientes cinco algoritmos de Machine Learning:

1. **Logistic Regression**
2. **Random Forest Classifier**
3. **XGBoost**
4. **Red Neuronal (Neural Network) con Keras**
5. **Red Neuronal (Neural Network) con PyTorch**

### Justificación de la Selección

1. **Logistic Regression**:
   - **Características**: Modelo lineal simple que es eficiente y fácil de interpretar.
   - **Justificación**: Adecuado para problemas de clasificación multiclase y sirve como un buen punto de referencia.
   - **Referencia**: [Hosmer, D.W., Lemeshow, S. (2000). Applied Logistic Regression. Wiley.]

2. **Random Forest Classifier**:
   - **Características**: Ensamble de árboles de decisión que maneja bien las interacciones no lineales y la importancia de características.
   - **Justificación**: Eficaz para manejar conjuntos de datos con características complejas y evita el sobreajuste.
   - **Referencia**: [Breiman, L. (2001). Random Forests. Machine Learning, 45(1), 5-32.]

3. **XGBoost**:
   - **Características**: Algoritmo de gradiente potenciado que es altamente eficiente y preciso, especialmente en problemas de clasificación y regresión.
   - **Justificación**: Excelente para manejar características de alta dimensionalidad y ofrece mecanismos avanzados de regularización para evitar el sobreajuste.
   - **Referencia**: [Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System. Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining.]

4. **Red Neuronal (Neural Network) con Keras**:
   - **Características**: Modelos capaces de capturar relaciones complejas y no lineales en los datos mediante múltiples capas de neuronas.
   - **Justificación**: Adecuada para procesamiento de texto y puede aprovechar representaciones de características profundas para mejorar la clasificación.
   - **Referencia**: [Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep Learning. MIT Press.]

5. **Red Neuronal (Neural Network) con PyTorch**:
   - **Características**: Similar a las redes neuronales con Keras, pero con mayor flexibilidad y control en el proceso de entrenamiento.
   - **Justificación**: Permite una personalización más detallada del modelo y el proceso de entrenamiento, lo que puede mejorar el rendimiento en tareas específicas.
   - **Referencia**: [Paszke, A., et al. (2019). PyTorch: An Imperative Style, High-Performance Deep Learning Library. Advances in Neural Information Processing Systems.]

---

In [24]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

## 2. Preprocesamiento e Ingeniería de Características

### Carga de Datos

In [25]:
import pandas as pd

# Cargar los datos preprocesados
train_df = pd.read_csv('data/train_preprocessed.csv')
test_df = pd.read_csv('data/test.csv')

# Visualizar las primeras filas del conjunto de entrenamiento
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,text_length,text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,317,hi im isaac im going writing face mars natural...
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,210,perspective think face natural landform dont t...
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,105,think face natural landform life mars descover...
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,362,life mars would know reason think natural land...
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,101,people thought face formed alieans thought lif...


### Limpieza de Datos

Aunque ya se ha realizado un limpieza previa, esta es solo una medida de seguridad.

In [26]:
# Verificar valores nulos
train_df.isnull().sum()

# Rellenar o eliminar valores nulos si es necesario
train_df = train_df.dropna()

### Ingeniería de Características (Feature Engineering)

1. **Tokenización y Normalización del Texto**:
   - Convertir texto a minúsculas.
   - Eliminar puntuación y caracteres especiales.
   - Tokenizar palabras.

   > Esto fue realizado en la etapa previa (Normalización)

2. **Vectorización**:
   - Utilizar TF-IDF para transformar el texto en vectores numéricos.

3. **Características Adicionales**:
   - Longitud del texto (`text_length`).
   - Número de palabras clave específicas relacionadas con elementos argumentativos.

### Codificación de Etiquetas (Label Encoding)

Antes de dividir los datos, es esencial codificar las etiquetas categóricas en valores numéricos para que los modelos de Scikit-Learn puedan procesarlas correctamente.

In [27]:
from sklearn.preprocessing import LabelEncoder

# Inicializar el codificador
le = LabelEncoder()

# Codificar las etiquetas de clase
y = le.fit_transform(train_df['discourse_effectiveness'])

# Opcional: Verificar la codificación
print(le.classes_)

['Adequate' 'Effective' 'Ineffective']


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# Vectorización del texto
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_text = tfidf.fit_transform(train_df['text'])

# Escalar la característica de longitud del texto
scaler = StandardScaler()
X_length = scaler.fit_transform(train_df[['text_length']])

# Concatenar características
import scipy.sparse as sp
X = sp.hstack([X_text, X_length])

### Justificación de las Técnicas Aplicadas

- **TF-IDF**: Captura la importancia relativa de las palabras en el contexto del corpus, lo que es esencial para entender la efectividad argumentativa.
- **Escalado de Características**: Normaliza la longitud del texto para que no domine otras características durante el entrenamiento del modelo.

---

## 3. Implementación de Modelos

### División de Datos

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Definición de Modelos

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Inicializar modelos tradicionales
log_reg = LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=42)
rand_forest = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Preparar datos para la Red Neuronal con Keras
# No es necesario re-codificar 'y_train' ya está codificada

# Definir el modelo de Red Neuronal con Keras
def create_nn_model(input_dim, num_classes):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

num_classes = len(le.classes_)
nn_model_keras = create_nn_model(X_train.shape[1], num_classes)
nn_model_keras.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Preparar datos para la Red Neuronal con PyTorch
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.toarray().astype('float32')
        self.y = y.astype('int64')

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset_pytorch = TextDataset(X_train, y_train)
val_dataset_pytorch = TextDataset(X_val, y_val)

train_loader_pytorch = DataLoader(train_dataset_pytorch, batch_size=128, shuffle=True)
val_loader_pytorch = DataLoader(val_dataset_pytorch, batch_size=128, shuffle=False)

# Definir el modelo de Red Neuronal con PyTorch
class NeuralNetPyTorch(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNetPyTorch, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

nn_model_pytorch = NeuralNetPyTorch(X_train.shape[1], num_classes)

### Entrenamiento y Validación Cruzada

#### Modelos Tradicionales

In [31]:
from sklearn.model_selection import cross_val_score

# Validación cruzada para Logistic Regression
log_reg_cv = cross_val_score(log_reg, X_train, y_train, cv=5, scoring='accuracy')
print(f'Logistic Regression CV Accuracy: {log_reg_cv.mean():.4f}')

# Validación cruzada para Random Forest
rand_forest_cv = cross_val_score(rand_forest, X_train, y_train, cv=5, scoring='accuracy')
print(f'Random Forest CV Accuracy: {rand_forest_cv.mean():.4f}')

# Validación cruzada para XGBoost
xgb_cv = cross_val_score(xgb, X_train, y_train, cv=5, scoring='accuracy')
print(f'XGBoost CV Accuracy: {xgb_cv.mean():.4f}')

Logistic Regression CV Accuracy: 0.6440
Random Forest CV Accuracy: 0.6384
XGBoost CV Accuracy: 0.6420


#### Red Neuronal con Keras

In [33]:
# Definir Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = nn_model_keras.fit(
    X_train.toarray(), y_train,
    epochs=50,
    batch_size=128,
    validation_data=(X_val.toarray(), y_val),
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9180 - loss: 0.2380 - val_accuracy: 0.6136 - val_loss: 1.2920
Epoch 2/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.9408 - loss: 0.1727 - val_accuracy: 0.6046 - val_loss: 1.4237
Epoch 3/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.9568 - loss: 0.1295 - val_accuracy: 0.6175 - val_loss: 1.5757
Epoch 4/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9634 - loss: 0.1062 - val_accuracy: 0.5958 - val_loss: 1.6708
Epoch 5/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9718 - loss: 0.0840 - val_accuracy: 0.6088 - val_loss: 1.7681
Epoch 6/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9741 - loss: 0.0783 - val_accuracy: 0.6026 - val_loss: 1.8860


#### Red Neuronal con PyTorch

In [34]:
# Definir el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nn_model_pytorch.to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(nn_model_pytorch.parameters(), lr=0.001)

# Entrenar el modelo
num_epochs = 50
early_stop_counter = 0
best_val_loss = float('inf')
patience = 5

for epoch in range(num_epochs):
    nn_model_pytorch.train()
    running_loss = 0.0
    for inputs, labels in train_loader_pytorch:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = nn_model_pytorch(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader_pytorch.dataset)

    # Validación
    nn_model_pytorch.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader_pytorch:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = nn_model_pytorch(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader_pytorch.dataset)
    val_accuracy = correct / total

    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.4f}')

    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        # Guardar el mejor modelo
        torch.save(nn_model_pytorch.state_dict(), 'best_model_pytorch.pth')
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping")
            break

# Cargar el mejor modelo
nn_model_pytorch.load_state_dict(torch.load('best_model_pytorch.pth'))

Epoch 1/50 - Train Loss: 0.8440 - Val Loss: 0.7883 - Val Acc: 0.6530
Epoch 2/50 - Train Loss: 0.7266 - Val Loss: 0.7949 - Val Acc: 0.6468
Epoch 3/50 - Train Loss: 0.6672 - Val Loss: 0.8153 - Val Acc: 0.6355
Epoch 4/50 - Train Loss: 0.6017 - Val Loss: 0.8580 - Val Acc: 0.6269
Epoch 5/50 - Train Loss: 0.5199 - Val Loss: 0.9311 - Val Acc: 0.6280
Epoch 6/50 - Train Loss: 0.4238 - Val Loss: 1.0382 - Val Acc: 0.6130
Early stopping


<All keys matched successfully>

### Optimización de Hiperparámetros

#### Modelos Tradicionales

In [36]:
from sklearn.model_selection import GridSearchCV

# Parámetros para Logistic Regression
param_grid_lr = {
    'C': [0.1, 1, 10],
    'solver': ['newton-cg', 'lbfgs', 'saga']
}

grid_lr = GridSearchCV(log_reg, param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_lr.fit(X_train, y_train)
print(f'Best Logistic Regression Params: {grid_lr.best_params_}')

# Parámetros para Random Forest
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_rf = GridSearchCV(rand_forest, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train)
print(f'Best Random Forest Params: {grid_rf.best_params_}')

# Parámetros para XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.1],
}

grid_xgb = GridSearchCV(xgb, param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_xgb.fit(X_train, y_train)
print(f'Best XGBoost Params: {grid_xgb.best_params_}')

Best Logistic Regression Params: {'C': 1, 'solver': 'lbfgs'}
Best Random Forest Params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
Best XGBoost Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}


---

## 4. Evaluación y Comparación de Modelos

### Evaluación del Rendimiento

In [38]:
from sklearn.metrics import classification_report, accuracy_score

# Predicciones
y_pred_lr = grid_lr.predict(X_val)
y_pred_rf = grid_rf.predict(X_val)
y_pred_xgb = grid_xgb.predict(X_val)

# Predicciones de la Red Neuronal con Keras
y_pred_nn_prob_keras = nn_model_keras.predict(X_val.toarray())
y_pred_nn_keras = y_pred_nn_prob_keras.argmax(axis=1)  # Mantener etiquetas numéricas

# Predicciones de la Red Neuronal con PyTorch
nn_model_pytorch.eval()
y_pred_nn_pytorch = []
with torch.no_grad():
    for inputs, _ in val_loader_pytorch:
        inputs = inputs.to(device)
        outputs = nn_model_pytorch(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred_nn_pytorch.extend(predicted.cpu().numpy())

# Ahora, y_pred_nn_pytorch ya está en formato numérico

# Reportes de clasificación
print("Logistic Regression Classification Report")
print(classification_report(y_val, y_pred_lr, target_names=le.classes_))

print("Random Forest Classification Report")
print(classification_report(y_val, y_pred_rf, target_names=le.classes_))

print("XGBoost Classification Report")
print(classification_report(y_val, y_pred_xgb, target_names=le.classes_))

print("Neural Network (Keras) Classification Report")
print(classification_report(y_val, y_pred_nn_keras, target_names=le.classes_))

print("Neural Network (PyTorch) Classification Report")
print(classification_report(y_val, y_pred_nn_pytorch, target_names=le.classes_))


230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Logistic Regression Classification Report
              precision    recall  f1-score   support

    Adequate       0.65      0.85      0.74      4211
   Effective       0.66      0.50      0.57      1873
 Ineffective       0.51      0.19      0.28      1268

    accuracy                           0.64      7352
   macro avg       0.61      0.51      0.53      7352
weighted avg       0.63      0.64      0.61      7352

Random Forest Classification Report
              precision    recall  f1-score   support

    Adequate       0.64      0.90      0.75      4211
   Effective       0.68      0.43      0.52      1873
 Ineffective       0.59      0.10      0.17      1268

    accuracy                           0.64      7352
   macro avg       0.64      0.48      0.48      7352
weighted avg       0.64      0.64      0.59      7352

XGBoost Classification Report
              precision    recall  f1-score   support

    Adequate       0.64      0.92 

---

## 5. Visualización y Comunicación de Resultados